In [ ]:
%pylab inline
import matplotlib.pyplot as plt

import os
import numpy as np
import pandas as pd


from importlib import reload
import sys
sys.path.append('../../code/scripts')
from dataset_params import dataset_params
import utils
reload(utils)
from utils import get_best_hp_results,read_subset_results_nonimage
import pickle
import itertools

import plotting


In [ ]:
# 0. setup
genre_pair = ['A', 'B']

# for plotting
genre_id_dict = {}
for k in range(2):
    genre_id_dict[k] = genre_pair[k]

In [ ]:
results_tag = 'no_demographics'
results_descriptor_ERM = 'mooc_subsetting_post_secondary_hpSel_{0}_ERM'.format(results_tag)
results_general_path = '../../results/subset_results/'
this_results_path_ERM = os.path.join(results_general_path, results_descriptor_ERM)

group_key = 'post_secondary'

#plotting
scale = 5
acc_key = 'auc_roc'

groups = [0,1]

gamma0 = dataset_params['mooc']['gamma']
gammas = [gamma0,1-gamma0]

# 1. Ridge regression

In [ ]:
reload(utils)
from utils import get_best_hp_results
pred_fxn_name = 'ridge'
param_dict = {'alpha': [0.001,0.01,0.1,1.0,10.0]}

pred_fxn_base_name = 'subset_{0}_5foldcv/{1}'.format(group_key, pred_fxn_name)
this_results_path_ERM = os.path.join(results_general_path, results_descriptor_ERM)
results_path_this_pred_fxn = os.path.join(this_results_path_ERM,pred_fxn_base_name)

print(param_dict)
rs_ERM = read_subset_results_nonimage(results_path_this_pred_fxn, param_dict)
rs_IW = read_subset_results_nonimage(results_path_this_pred_fxn.replace('ERM','IW'), param_dict)

opt_df_ERM, _ = get_best_hp_results(results_path_this_pred_fxn,param_dict, 
                                    gammas=gammas,acc_key = acc_key)
opt_df_IW, _ = get_best_hp_results(results_path_this_pred_fxn.replace('ERM','IW'),
                                  param_dict, gammas=gammas,acc_key = acc_key)

alphas = param_dict['alpha']

fig, ax = plt.subplots(1,len(alphas),
                       figsize=(scale*len(alphas), scale*1),
                       sharey=True)

rs_dicts = [rs_ERM, rs_IW]
objs = ['ERM','IW']
ls = ['-','--' ]
for i,alpha in enumerate(alphas):
    for j,rs in enumerate(rs_dicts):
        results = rs[(alpha,)]
        cv_accs_by_group, cv_accs_total = results['accs_by_group'], results['accs_total']
        subset_sizes_cv = results['subset_sizes']
        subset_fracs_cv = subset_sizes_cv / subset_sizes_cv.sum(axis=0)
        plotting.plot_by_group(cv_accs_by_group,
                               cv_accs_total,
                               subset_fracs_cv,
                               acc_key,
                               genre_id_dict,
                              # gammas = [gamma0,1-gamma0],
                               pop_weights = 'by_eval_set',
                               label_append = ' ({0})'.format(objs[j]),
                               range_type='stddev', 
                               title='alpha: {0}'.format(alpha),
                               ax=ax[i],
                               ls = ls[j],
                               ylim=None,
                               legend=False)
    
plt.legend();
plt.ylim(0.92,1.0)

In [ ]:
opt_df_ERM[opt_df_ERM['group'] == 'min_over_groups']

# 2. logistic regression

In [ ]:
reload(utils)
reload(plotting)
from utils import get_best_hp_results,read_subset_results_nonimage

pred_fxn_name = 'logistic_regression'
param_dict = {'penalty': ['l2'],
              'C': [0.001, 0.01, 0.1,1.0, 10.0],
              'solver': ['lbfgs']
             }

pred_fxn_base_name = 'subset_{0}_5foldcv/{1}'.format(group_key, pred_fxn_name)
this_results_path_ERM = os.path.join(results_general_path, results_descriptor_ERM)
results_path_this_pred_fxn = os.path.join(this_results_path_ERM,pred_fxn_base_name)

rs_ERM = read_subset_results_nonimage(results_path_this_pred_fxn, param_dict)
rs_IW = read_subset_results_nonimage(results_path_this_pred_fxn.replace('ERM','IW'), param_dict)

opt_df_ERM, _ = get_best_hp_results(results_path_this_pred_fxn,param_dict, 
                                    gammas=gammas,acc_key = acc_key)
opt_df_IW, _ = get_best_hp_results(results_path_this_pred_fxn.replace('ERM','IW'),
                                  param_dict, gammas=gammas,acc_key = acc_key)

penalties = param_dict['penalty']
cs = param_dict['C']
solver = param_dict['solver'][0]

fig, ax = plt.subplots(len(penalties),len(cs),
                       figsize=(scale*len(cs),scale*len(penalties)),
                       sharey=True)

results_dicts = [rs_ERM, rs_IW]
objs = ['ERM','IW']
ls = ['-','--' ]

for j,penalty in enumerate(penalties):
    for i,c in enumerate(cs):
        for k,rs in enumerate(results_dicts):
            results = rs[(penalty, c, solver)]
            cv_accs_by_group, cv_accs_total = results['accs_by_group'], results['accs_total']
            subset_sizes_cv = results['subset_sizes']
            subset_fracs_cv = subset_sizes_cv / subset_sizes_cv.sum(axis=0)
            plotting.plot_by_group(cv_accs_by_group,
                                   cv_accs_total,
                                   subset_fracs_cv,
                                   acc_key,
                                   genre_id_dict,
                                 #  gammas = gammas,
                                   pop_weights = 'by_eval_set',
                                   label_append = ' ({0})'.format(objs[k]),
                                   range_type='stddev', 
                                   title='C: {0}, pen: {1}'.format(c,penalty),
                                   ls = ls[k],
                                   ax=ax[i],
                                   ylim=None,
                                   legend=False)
        
    ax[i].legend()
plt.ylim(0.92,1.0)

In [ ]:
opt_df_ERM[opt_df_ERM['group'] == 'gamma_avged']


# 3. random forest classifier

In [ ]:
rs = {}
pred_fxn_name = 'rf_classifier'

if results_tag == 'no_demographics':
    max_depths = [4, 8, 16, 32]
else:
    max_depths = [8, 16, 32]
param_dict = {'max_depth': max_depths,
              'n_estimators': [100,200,400]
              }

pred_fxn_base_name = 'subset_{0}_5foldcv/{1}'.format(group_key, pred_fxn_name)
this_results_path_ERM = os.path.join(results_general_path, results_descriptor_ERM)
results_path_this_pred_fxn = os.path.join(this_results_path_ERM,pred_fxn_base_name)

rs_ERM = read_subset_results_nonimage(results_path_this_pred_fxn, param_dict)
rs_IW = read_subset_results_nonimage(results_path_this_pred_fxn.replace('ERM','IW'), param_dict)

opt_df_ERM, _ = get_best_hp_results(results_path_this_pred_fxn,param_dict, 
                                    gammas=gammas,acc_key = acc_key)
opt_df_IW, _ = get_best_hp_results(results_path_this_pred_fxn.replace('ERM','IW'),
                                  param_dict, gammas=gammas,acc_key = acc_key)

max_depths = param_dict['max_depth']
n_estimators = param_dict['n_estimators']

scale = 5
fig, ax = plt.subplots(len(max_depths),len(n_estimators),
                       figsize=(scale*len(n_estimators), scale*len(max_depths)),
                       sharey=True)

rs_dicts = [rs_ERM, rs_IW]
objs = ['ERM','IW']
ls = ['-','--' ]


for j,max_depth in enumerate(max_depths):
    for i,n_estimator in enumerate(n_estimators):
        for k, rs in enumerate(rs_dicts):
                results = rs[(max_depth, n_estimator)]
                cv_accs_by_group, cv_accs_total = results['accs_by_group'], results['accs_total']
                subset_sizes_cv = results['subset_sizes']
                subset_fracs_cv = subset_sizes_cv / subset_sizes_cv.sum(axis=0)
                plotting.plot_by_group(cv_accs_by_group,
                                       cv_accs_total,
                                       subset_fracs_cv,
                                       acc_key,
                                       genre_id_dict,
                                               label_append = ' ({0})'.format(objs[k]),
                                               title='max_depth: {0}, n_estimators: {1}'.format(max_depth,n_estimator),
                                               ax=ax[j,i],
                                               ls = ls[k],
                                               ylim=None,
                                               legend=False)
                ax[j,i].legend()
                

In [ ]:
opt_df_ERM[opt_df_ERM['group'] == 'pop']

In [ ]:
opt_df_IW[opt_df_IW['group'] == 'pop']

In [ ]:
opt_df_ERM[opt_df_ERM['group'] == 1]

In [ ]:
opt_df_ERM[opt_df_ERM['group'] == 'gamma_avged']